In [1]:
import pandas as pd

In [23]:
rows = []
with open("../data/tripadvisor_restaurant/Ha_Noi_with_links.csv", encoding="utf-8") as f:
    next(f)  # skip header
    for line in f:
        parts = line.strip().split(',', 1)  # split only on the first comma
        if len(parts) == 2:
            rows.append(parts)

df = pd.DataFrame(rows, columns=["restaurant_id", "restaurant_link"])
df.head()

,restaurant_id,restaurant_link
0,7321100,https://www.google.com/maps/place/Banh+Mi+25/@...
1,1124698,https://www.google.com/maps/place/Quan+An+Ngon...
2,12793763,https://www.google.com/maps/place/Hong+Hoai's+...
3,19255094,https://www.google.com/maps/place/Hoang's+Rest...
4,2548208,https://www.google.com/maps/place/Essence+Rest...


In [24]:
df.shape

(1000, 2)

In [26]:
def extract_lat_lon_from_url(url):
    """
    Extract latitude and longitude from a Google Maps URL.
    Returns (lat, lon) as strings, or (None, None) if not found.
    """
    try:
        # Example: .../@21.036118,105.8460021,17z/...
        at_part = url.split('/@')[1].split('/')[0]
        lat, lon = at_part.split(',')[:2]
        return lat, lon
    except Exception:
        return None, None

# Example usage:
df[['lat', 'lon']] = df['restaurant_link'].apply(lambda url: pd.Series(extract_lat_lon_from_url(url)))
df.head()

,restaurant_id,restaurant_link,lat,lon
0,7321100,https://www.google.com/maps/place/Banh+Mi+25/@...,21.036118,105.8460021
1,1124698,https://www.google.com/maps/place/Quan+An+Ngon...,21.0263067,105.8408014
2,12793763,https://www.google.com/maps/place/Hong+Hoai's+...,21.0338489,105.8448976
3,19255094,https://www.google.com/maps/place/Hoang's+Rest...,21.0358972,105.8489602
4,2548208,https://www.google.com/maps/place/Essence+Rest...,21.0306464,105.8392623


In [33]:
data =df

In [34]:
# Check for duplicates based on 'restaurant_id' and remove them
data = data.drop_duplicates(subset='restaurant_id', keep='first').reset_index(drop=True)
data

,restaurant_id,restaurant_link,lat,lon
0,7321100,https://www.google.com/maps/place/Banh+Mi+25/@...,21.036118,105.8460021
1,1124698,https://www.google.com/maps/place/Quan+An+Ngon...,21.0263067,105.8408014
2,12793763,https://www.google.com/maps/place/Hong+Hoai's+...,21.0338489,105.8448976
3,19255094,https://www.google.com/maps/place/Hoang's+Rest...,21.0358972,105.8489602
4,2548208,https://www.google.com/maps/place/Essence+Rest...,21.0306464,105.8392623
...,...,...,...,...
995,32994333,https://www.google.com/maps/place/AMY+Indian+R...,None,None
996,5277948,https://www.google.com/maps/place/Zpizza/data=...,None,None
997,26802639,https://www.google.com/maps/place/Pizza+4P%E2%...,None,None
998,11707989,https://www.google.com/maps/place/Takumi+Resta...,21.0330595,105.8072619


In [37]:
data_2 = pd.read_csv('../data/tripadvisor_restaurant/Ha Noi.csv')
data_2.shape

(1000, 32)

In [54]:
# data_2

In [55]:
# data_2.head()

In [51]:
data['restaurant_id'] = data['restaurant_id'].astype(str)
data_2['restaurant_id'] = data_2['restaurant_id'].astype(str)

In [ ]:
data['restaurant_id'] = data['restaurant_id'].astype(str)
data_2['restaurant_id'] = data_2['restaurant_id'].astype(str)

# Drop rows with missing lat/lon
data = data.dropna(subset=['lat', 'lon']).reset_index(drop=True)

merged = pd.merge(
    data,
    data_2[['restaurant_id', 'latitude', 'longitude']],
    on='restaurant_id',
    how='inner'
)

# Calculate the absolute difference
merged['lat_diff'] = (merged['lat'] - merged['latitude']).abs()
merged['lon_diff'] = (merged['lon'] - merged['longitude']).abs()

# Set a threshold for "too much different" (e.g., 0.01 degrees ~1km)
threshold = 0.01
filtered = merged[(merged['lat_diff'] <= threshold) & (merged['lon_diff'] <= threshold)].reset_index(drop=True)

# If you want only the filtered restaurant_ids:
filtered_restaurant_ids = filtered['restaurant_id'].tolist()

# Or, if you want to filter the original data DataFrames:
data = data[data['restaurant_id'].isin(filtered_restaurant_ids)].reset_index(drop=True)
data_2 = data_2[data_2['restaurant_id'].isin(filtered_restaurant_ids)].reset_index(drop=True)

data.head()

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [19]:
data.to_csv('../data/tripadvisor_restaurant/restaurant_links1.csv', index=False, encoding='utf-8-sig')